# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/albert/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/albert/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/albert/Development/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/albert/Development/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/albert/Development/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/27 [00:00<?, ?it/s]

Property 'summary' already exists in node '7a615d'. Skipping!
Property 'summary' already exists in node 'b8be25'. Skipping!
Property 'summary' already exists in node '3bca42'. Skipping!
Property 'summary' already exists in node '41447c'. Skipping!
Property 'summary' already exists in node 'efccd9'. Skipping!
Property 'summary' already exists in node '77596c'. Skipping!
Property 'summary' already exists in node 'ead53c'. Skipping!
Property 'summary' already exists in node '21d0be'. Skipping!
Property 'summary' already exists in node 'bb0165'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/59 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'efccd9'. Skipping!
Property 'summary_embedding' already exists in node '3bca42'. Skipping!
Property 'summary_embedding' already exists in node '7a615d'. Skipping!
Property 'summary_embedding' already exists in node 'bb0165'. Skipping!
Property 'summary_embedding' already exists in node '21d0be'. Skipping!
Property 'summary_embedding' already exists in node 'b8be25'. Skipping!
Property 'summary_embedding' already exists in node '77596c'. Skipping!
Property 'summary_embedding' already exists in node 'ead53c'. Skipping!
Property 'summary_embedding' already exists in node '41447c'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 45, relationships: 388)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 45, relationships: 388)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms

##### ✅ Answer:
* SingleHopSpecificQuerySynthesizer: Generate simple and direct questions that only need a concrete fact or a short sentenc to be answered.
* MultiHopAbstractQuerySynthesizer: These questions needs more reasoning and understanding of the concepts asked. So, it is possible that the LLM needs some cycles to answer.
* MultiHopSpecificQuerySynthesizer: These are specific questions that to be answered the LLM needs to pull info from different sources and understand it.




Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Can you explain what Federal Student Aid encom...,[Application and Verification Guide Introducti...,Federal Student Aid refers to the financial ai...,single_hop_specifc_query_synthesizer
1,What is FAFSA?,[Chapter 1: The Application Process We removed...,FAFSA is the application process for federal a...,single_hop_specifc_query_synthesizer
2,What is the FSa Knowlege Center?,[electronically due to limitations on access t...,The FSA Knowledge Center is a resource page th...,single_hop_specifc_query_synthesizer
3,Can you tell me about the 2024-25 FAFSA and ho...,[Note: There will be two active portals that c...,Note: There will be two active portals that co...,single_hop_specifc_query_synthesizer
4,What does FPS do in the FAFSA process?,[The FPS also checks the application for possi...,The FPS checks the application for possible in...,single_hop_specifc_query_synthesizer
5,"How do comment codes, C Flags, and resolution ...",[<1-hop>\n\nOutput Documents After processing ...,"Comment codes, C Flags, and resolution require...",multi_hop_abstract_query_synthesizer
6,How do the role of financial aid administrator...,[<1-hop>\n\nApplication and Verification Guide...,The role of financial aid administrators and c...,multi_hop_abstract_query_synthesizer
7,How does the eligibility for National Guard an...,[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,The context explains that members of the Natio...,multi_hop_abstract_query_synthesizer
8,How does the 2025-26 FAFSA process incorporate...,[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,The 2025-26 FAFSA process aligns the veteran d...,multi_hop_specific_query_synthesizer
9,How does the FSA help students with the FAFSA ...,[<1-hop>\n\nApplication and Verification Guide...,The FSA provides guidance to college financial...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/29 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c89511'. Skipping!
Property 'summary' already exists in node 'dff13e'. Skipping!
Property 'summary' already exists in node '846e28'. Skipping!
Property 'summary' already exists in node '708b0b'. Skipping!
Property 'summary' already exists in node '61ce8a'. Skipping!
Property 'summary' already exists in node '154bd1'. Skipping!
Property 'summary' already exists in node 'a207a6'. Skipping!
Property 'summary' already exists in node '7b0a78'. Skipping!
Property 'summary' already exists in node 'a7ae79'. Skipping!
Property 'summary' already exists in node '68e62a'. Skipping!
Property 'summary' already exists in node '05c406'. Skipping!


...

Property 'summary_embedding' already exists in node 'a207a6'. Skipping!
Property 'summary_embedding' already exists in node '154bd1'. Skipping!
Property 'summary_embedding' already exists in node '846e28'. Skipping!
Property 'summary_embedding' already exists in node '7b0a78'. Skipping!
Property 'summary_embedding' already exists in node '05c406'. Skipping!
Property 'summary_embedding' already exists in node '61ce8a'. Skipping!
Property 'summary_embedding' already exists in node 'c89511'. Skipping!
Property 'summary_embedding' already exists in node '68e62a'. Skipping!
Property 'summary_embedding' already exists in node '708b0b'. Skipping!
Property 'summary_embedding' already exists in node 'a7ae79'. Skipping!
Property 'summary_embedding' already exists in node 'dff13e'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Federal Student Aid?,[Application and Verification Guide Introducti...,Federal Student Aid refers to the financial ai...,single_hop_specifc_query_synthesizer
1,Can you tell me about the recent updates or ch...,[Chapter 1: The Application Process We removed...,Chapter 1: The Application Process We removed ...,single_hop_specifc_query_synthesizer
2,What is FAFSA Submission Summary?,[Output Documents After processing is complete...,The FAFSA Submission Summary is an output docu...,single_hop_specifc_query_synthesizer
3,Could you please explain the significance of a...,[Valid Output Document 34 CFR 668.2(b)],Valid Output Document 34 CFR 668.2(b) is refer...,single_hop_specifc_query_synthesizer
4,How do the authorization and consent processes...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The authorization and consent processes for FT...,multi_hop_abstract_query_synthesizer
5,how parent support impact on dependnt status a...,"[<1-hop>\n\nveteran, or will be one by June 30...",The context explains that students with legal ...,multi_hop_abstract_query_synthesizer
6,Wht roles of the dept and partners in handling...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The department is responsible for disclosing F...,multi_hop_abstract_query_synthesizer
7,"How do the legal references such as Title 38, ...",[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,"The legal references, including Title 38 of th...",multi_hop_abstract_query_synthesizer
8,How do the ISIR and FAFSA Submission Summary r...,[<1-hop>\n\nOutput Documents After processing ...,The ISIR is an output document produced after ...,multi_hop_specific_query_synthesizer
9,"Considering the FAFSA deadline of June 30, 202...",[<1-hop>\n\nequal the tax filer(s) plus depend...,The eligibility criteria for legal dependents ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [19]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [20]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [21]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [24]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [26]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [29]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loan  \n- Direct Unsubsidized Loan  \n- Direct PLUS Loan (student Federal PLUS Loan and parent PLUS Loan on behalf of a dependent student)  \n- Subsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)  \n- Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)  \n- Federal SLS Loans (made under the FFEL Program before July 1, 2010)  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)\n\nAdditionally, some loans mentioned include Direct Consolidation Loans or Federal Consolidation Loans (consolidation loans under the FFEL Program).  \n\nDirect Subsidized Loans are only available to undergraduate students, while graduate or professional students are eligible only for Direct Unsubsidized Loans and Direct PLUS Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [30]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [31]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

- `qa_evaluator`: the correctness of teh model's answer vs the reference answer. Matches the response with the expected respone.
- `labeled_helpfulness_evaluator`: Measures the helpfulness using the reference answer as reference. Basically, it checks if the answer provides value.
- `empathy_evaluator`: Checks the empathy, cheks if the response for the user fell heard or supported.

## LangSmith Evaluation

In [32]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'complicated-advertising-10' at:
https://smith.langchain.com/o/414e1faa-44a7-4b58-9326-1d7c5ce70c12/datasets/5727e542-93d0-41e0-9eab-0e50347ec09f/compare?selectedSessions=e3d34f2e-a11a-423b-ad5b-1a9771582de9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"According to the definitions provided, how doe...","According to the provided context, a student c...",None,The context explains that persons on active du...,1,1,0,10.300112,8daff935-ecf4-447a-a95f-ab93212bd5ab,9024b832-a2f1-4d1a-b5ed-631e08320e64
1,How does the FA-DDX improve the verification p...,The FA-DDX improves the verification process f...,None,The FA-DDX enhances the verification process b...,1,1,0,2.034341,5e34663f-141f-4930-a319-26832283a10e,0aee6be0-a241-4ce3-b006-7a771729023a
2,"Considering the FAFSA deadline of June 30, 202...",Based on the context provided:\n\nThe FAFSA ap...,None,The eligibility criteria for legal dependents ...,1,1,0,4.625460,a7a97687-6b7f-42e4-9a2c-d2f2e1ea7b26,c691a99e-12a0-4e6b-8b68-6e4f9eb785a3
3,How do the ISIR and FAFSA Submission Summary r...,The FAFSA Submission Summary and the ISIR are ...,None,The ISIR is an output document produced after ...,1,1,0,3.560958,61ad1299-9f6c-40c3-b592-b85a76339675,e1fae9ef-8b34-42c5-8287-33d122d31572
4,"How do the legal references such as Title 38, ...",The legal references such as 38 USC 101 and 38...,None,"The legal references, including Title 38 of th...",1,1,0,2.060159,42bb3cbe-5f4b-46a7-abd2-3e70bef28cd6,b3fa8c0b-1d1d-4e13-aeb6-403e1b570d36
5,Wht roles of the dept and partners in handling...,Based on the provided context:\n\n- The Depart...,None,The department is responsible for disclosing F...,1,1,0,2.150572,798c9d03-ea27-4153-a8b5-0b95d4e08b81,df0beb81-a725-4a89-8da5-252432d712c3
6,how parent support impact on dependnt status a...,Parent support impacts a student's dependency ...,None,The context explains that students with legal ...,1,1,0,2.737057,cdb0f3ca-89f3-4c53-afca-d2b1e32b0ff9,7a838d11-a157-4294-a219-6cc922cbd9b1
7,How do the authorization and consent processes...,The authorization and consent processes for Fe...,None,The authorization and consent processes for FT...,1,1,0,7.462987,01055620-7d8f-4d6f-96c2-0d6c481d4651,33367d2f-ecc2-4fd0-95b2-0ef2264f9f32
8,Could you please explain the significance of a...,"A Valid Output Document, as outlined in 34 CFR...",None,Valid Output Document 34 CFR 668.2(b) is refer...,1,0,0,1.646035,13fd0eff-cc76-4094-b3f0-314e0cd3b3b5,231a8e08-c2f3-4c55-82c2-9124a6b87c33
9,What is FAFSA Submission Summary?,The FAFSA Submission Summary is a document mad...,None,The FAFSA Submission Summary is an output docu...,1,1,0,1.965494,1295cba8-4211-4034-9b46-332771643589,677c3390-c8cf-44db-9cf2-9bc9e6d59806


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [33]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [34]:
rag_documents = docs

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

It is important becuase it directly affects how the source documents used to answer and give knowledge are splitted. Basically it changes the context of the RAG, and we want our system to just have the necessary and concrete context.
For example, If the chunks are too small maybe we are missing info in each chunk and there are incomplete. In other case, if the chunks are too big, maybe we are putting a lot of info that doesn't matter to answer the questions.

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

Because the embedding model is the main responsible on converting the chunks into vectors. In our case, if we want chunks with richer context and quality on the embedding having a "better" model can help on creating these vectors with more quality and afterwars, the retriever or other pieces that uses the vectors can be better on the retrieval and processing flows.

In [37]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [38]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it’s really important to understand the types of loans available to help manage educational expenses. Based on the context provided, there are a few kinds of loans available:\n\n1. **Direct Subsidized Loans:** These are loans where the amount you can borrow is based on your financial need (calculated as the Cost of Attendance minus other aid). Interest on these loans is paid by the government while you’re in school at least half-time.\n\n2. **Direct Unsubsidized Loans:** These loans are available regardless of financial need, and you, as the borrower, are responsible for the interest during all periods.\n\n3. **Direct PLUS Loans:** Parents of dependent students (or graduate/professional students) may take out these loans to help pay for the student’s educational costs, as long as they meet eligibility requirements. There isn’t a fixed limit on these loans, but they can’t exceed the student’s Cost of Attendance minus other aid received.\n\nAlso, if a depende

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'advanced-history-3' at:
https://smith.langchain.com/o/414e1faa-44a7-4b58-9326-1d7c5ce70c12/datasets/5727e542-93d0-41e0-9eab-0e50347ec09f/compare?selectedSessions=d8db2588-b938-435a-953e-e163128cf6df




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"According to the definitions provided, how doe...",Thank you for your question—it's important to ...,None,The context explains that persons on active du...,1,1,1,4.975798,8daff935-ecf4-447a-a95f-ab93212bd5ab,15d0f518-ccf7-4414-aab2-46a29f57147c
1,How does the FA-DDX improve the verification p...,Thank you for your thoughtful question. From w...,None,The FA-DDX enhances the verification process b...,1,1,1,5.593358,5e34663f-141f-4930-a319-26832283a10e,02b48696-2ad0-449b-878c-968772fd13d1
2,"Considering the FAFSA deadline of June 30, 202...",Thank you for your thoughtful question. It sou...,None,The eligibility criteria for legal dependents ...,1,1,1,5.871074,a7a97687-6b7f-42e4-9a2c-d2f2e1ea7b26,f70bb7f0-7431-48a0-8b79-1dbd67bdf94d
3,How do the ISIR and FAFSA Submission Summary r...,Thank you for your thoughtful question. It sou...,None,The ISIR is an output document produced after ...,1,1,1,4.234904,61ad1299-9f6c-40c3-b592-b85a76339675,f9f62812-c004-4f0b-8312-7370621e7718
4,"How do the legal references such as Title 38, ...",Thank you for reaching out with your question—...,None,"The legal references, including Title 38 of th...",1,1,1,6.475625,42bb3cbe-5f4b-46a7-abd2-3e70bef28cd6,5f786c5c-6e3b-486e-82be-05090d8e8fc1
5,Wht roles of the dept and partners in handling...,Thank you for your thoughtful question. Based ...,None,The department is responsible for disclosing F...,1,1,1,4.382043,798c9d03-ea27-4153-a8b5-0b95d4e08b81,c335ca9e-2dfe-4e87-87e7-8a46812ce353
6,how parent support impact on dependnt status a...,Thank you for your thoughtful question. From t...,None,The context explains that students with legal ...,1,1,1,4.291270,cdb0f3ca-89f3-4c53-afca-d2b1e32b0ff9,9bae69e1-8994-4dab-b26d-53419e607b6e
7,How do the authorization and consent processes...,Thank you for your thoughtful question about t...,None,The authorization and consent processes for FT...,1,1,1,7.810675,01055620-7d8f-4d6f-96c2-0d6c481d4651,caa2fd45-5760-40bd-94bc-e89647bc4aa2
8,Could you please explain the significance of a...,Thank you for your thoughtful question. I unde...,None,Valid Output Document 34 CFR 668.2(b) is refer...,1,1,1,4.401552,13fd0eff-cc76-4094-b3f0-314e0cd3b3b5,7d46b974-cb0f-4fbb-aa4d-18644d9f318a
9,What is FAFSA Submission Summary?,Thank you for your question! Based on the info...,None,The FAFSA Submission Summary is an output docu...,1,0,1,3.059350,1295cba8-4211-4034-9b46-332771643589,cb635526-305f-4530-aeda-e6ecafa74c34


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:

Experiment 1:

[]